<a href="https://colab.research.google.com/github/voilszee/kelompok-11-apm/blob/main/fpgrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.2 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [2]:
#Import dataset
from google.colab import files
uploaded = files.upload()

Saving groceries.csv to groceries.csv


In [3]:
#Import library
import numpy as np
import pandas as pd
import plotly.express as px
import io
from mlxtend.frequent_patterns import fpgrowth

In [4]:
dataset = pd.read_csv(io.BytesIO(uploaded['groceries.csv']), encoding= 'latin-1')
data = dataset[['Item 1','Item 2','Item 3']]
print(data)

                   Item 1               Item 2            Item 3
0            citrus fruit  semi-finished bread         margarine
1          tropical fruit               yogurt            coffee
2              whole milk                  NaN               NaN
3               pip fruit               yogurt      cream cheese
4        other vegetables           whole milk    condensed milk
...                   ...                  ...               ...
9830              sausage              chicken              beef
9831    cooking chocolate                  NaN               NaN
9832              chicken         citrus fruit  other vegetables
9833  semi-finished bread        bottled water              soda
9834              chicken       tropical fruit  other vegetables

[9835 rows x 3 columns]


In [5]:
data.shape

(9835, 3)

In [7]:
#Menyimpan item ke dalam array NumPy
item = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        item.append(data.values[i,j])

item = np.array(item)
print(item)

['citrus fruit' 'semi-finished bread' 'margarine' ... 'chicken'
 'tropical fruit' 'other vegetables']


In [8]:
#Menghapus nilai null dan menampilkan 5 item teratas
df = pd.DataFrame(item, columns=["items"]) 

df["incident_count"] = 1 

indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)
 
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

df_table.head(5).style.background_gradient(cmap='Blues')

,items,incident_count
0,whole milk,1877
1,other vegetables,1425
2,rolls/buns,1007
3,tropical fruit,980
4,sausage,924


In [9]:
#Visualisasi 50 item teratas dengan TreeMap
df_table["all"] = "50 Item Terfavorit" 

fig = px.treemap(df_table.head(50), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(50), hover_data=['items'],
                  color_continuous_scale='Blues',
                )
fig.show()

In [10]:
#Preprocessing dataset
transaction = []
for i in range(df.shape[0]):
    transaction.append([str(df.values[i,j]) for j in range(df.shape[1])])

transaction = np.array(transaction)

from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head()

/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional inform

,1,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [11]:
#Memilih 30 item teratas
first30 = df_table["items"].head(30).values 

df = df.loc[:,first30] 

df.shape

(23544, 30)

In [12]:
#Implementasi algoritma fpgrowth
res=fpgrowth(df,min_support=0.01, use_colnames=True)

res.head(10)

,support,itemsets
0,0.033087,(citrus fruit)
1,0.041624,(tropical fruit)
2,0.033172,(yogurt)
3,0.010151,(coffee)
4,0.079723,(whole milk)
5,0.027820,(pip fruit)
6,0.060525,(other vegetables)
7,0.012572,(butter)
8,0.042771,(rolls/buns)
9,0.021322,(beef)


In [13]:
#Membuat association rules dari elemen yang sering muncul
from mlxtend.frequent_patterns import association_rules

nres=association_rules(res, metric="lift", min_threshold=0.01)

nres

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
